# Functional goals for 2019 Q2

This notebook allows interactive exploration of the functionality targeted for the GROMACS master branch in 2019 Q2. 

For named features `fr0`, `fr1`, etcetera, you can build the `acceptance` docker image or download it from docker hub and run with, for example, `docker run --rm -p 8888:8888 gmxapi/acceptance:fr1`

Note that there isn't a great way to use a Jupyter notebook as the front-end to an MPI job. The notebook can be converted to a script and run non-interactively.

    jupyter nbconvert RequiredFunctionality.ipynb --to python
    python RequiredFunctionality.py
    # or
    # mpiexec -n 2 python -m mpi4py RequiredFunctionality.py

Before committing changes to this notebook, clear the output and/or run `python strip_notebook.py RequiredFunctionality.py`

Cells with unimplemented features will raise `gmxapi.exceptions.FeatureError`,
so an automated test can catch the first occurence of this exception and exit cleanly.